<a href="https://colab.research.google.com/github/Vardhinedi5869ms/PoC/blob/main/Predictive_Maintenance_Solutions_PoC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.4 MB/s eta 0:00:00


In [3]:
!pip install tensorflow

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Layer, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import gradio as gr
import plotly.graph_objects as go
import tensorflow as tf
import traceback

# Custom Attention Layer with Multi-Output
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight',
                                 shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(name='attention_bias',
                                 shape=(input_shape[1], 1),
                                 initializer='zeros',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        e = tf.tanh(tf.tensordot(inputs, self.W, axes=[-1, 0]) + self.b)
        alpha = tf.nn.softmax(e, axis=1)
        context = inputs * alpha
        context = tf.reduce_sum(context, axis=1)
        context = tf.expand_dims(context, axis=1)
        return context, alpha

    def compute_output_shape(self, input_shape):
        return [(input_shape[0], 1, input_shape[-1]), (input_shape[0], input_shape[1], 1)]

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Simulate realistic engine sensor data
n_samples = 5000
timesteps = 100
n_features = 10

def generate_realistic_data():
    data = np.zeros((n_samples, timesteps, n_features))
    ttf = np.zeros(n_samples)
    for i in range(n_samples):
        params = [
            np.random.uniform(5, 15),    # Vibration_Hz
            np.random.uniform(1200, 1800),  # Engine_Thrust_kN
            np.random.uniform(90, 130),  # Fuel_Pump_Pressure_bar
            np.random.uniform(30, 60),   # Temperature_C
            0,                          # Operating_Hours
            np.random.uniform(9000, 11000),  # Turbine_Speed_rpm
            np.random.uniform(2.5, 3.5),  # Oil_Pressure_bar
            np.random.uniform(850, 950),  # Exhaust_Gas_Temp_C
            np.random.uniform(25, 35),   # Fuel_Flow_Rate_Ls
            np.random.uniform(250, 350)   # Structural_Stress_MPa
        ]
        failure_point = np.random.randint(120, 250)
        for t in range(timesteps):
            if t < failure_point:
                params[0] += np.random.uniform(0, 0.2)
                params[1] -= np.random.uniform(0, 3)
                params[2] += np.random.uniform(-0.5, 0.5)
                params[3] += np.random.uniform(0, 0.2)
                params[4] += 10
                params[5] -= np.random.uniform(0, 20)
                params[6] -= np.random.uniform(0, 0.005)
                params[7] += np.random.uniform(0, 2)
                params[8] -= np.random.uniform(0, 0.05)
                params[9] += np.random.uniform(0, 0.5)
            else:
                params[0] = min(params[0] + np.random.uniform(1, 3), 40)
                params[1] = max(params[1] - np.random.uniform(10, 30), 600)
                params[2] += np.random.uniform(-10, 10)
                params[3] = min(params[3] + np.random.uniform(1, 5), 90)
                params[4] += 10
                params[5] = max(params[5] - np.random.uniform(50, 100), 6000)
                params[6] = max(params[6] - np.random.uniform(0.05, 0.2), 1.5)
                params[7] = min(params[7] + np.random.uniform(10, 30), 1100)
                params[8] = max(params[8] - np.random.uniform(0.5, 2), 15)
                params[9] = min(params[9] + np.random.uniform(2, 5), 450)

            data[i, t] = params[:]
        ttf[i] = max(0, (failure_point - timesteps) * 10)
    return data, ttf

# Generate and preprocess data
X, y = generate_realistic_data()
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler(feature_range=(0, 2000))
X_scaled = np.array([scaler_X.fit_transform(x) for x in X])
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Train-validation-test split
train_size = int(n_samples * 0.7)
val_size = int(n_samples * 0.15)
X_train, X_val, X_test = X_scaled[:train_size], X_scaled[train_size:train_size+val_size], X_scaled[train_size+val_size:]
y_train, y_val, y_test = y_scaled[:train_size], y_scaled[train_size:train_size+val_size], y_scaled[train_size+val_size:]

# Model 1: LSTM with Attention
inputs = Input(shape=(timesteps, n_features))
lstm1 = LSTM(128, return_sequences=True)(inputs)
dropout1 = Dropout(0.3)(lstm1)
attention_output = AttentionLayer(name='attention_layer')(dropout1)
lstm2 = LSTM(64)(attention_output[0])
dropout2 = Dropout(0.3)(lstm2)
dense1 = Dense(64, activation='relu')(dropout2)
outputs = Dense(1, activation='linear')(dense1)

model_with_attention = Model(inputs=inputs, outputs=[outputs, attention_output[1]])
model_with_attention.compile(optimizer=Adam(learning_rate=0.0001),
                            loss=['mse', 'mse'],
                            loss_weights=[1.0, 0.0])
model_with_attention.fit(X_train, [y_train, np.zeros((len(y_train), timesteps, 1))], epochs=150, batch_size=64,
                        validation_data=(X_val, [y_val, np.zeros((len(y_val), timesteps, 1))]),
                        callbacks=[EarlyStopping(patience=20, restore_best_weights=True)], verbose=0)

# Model 2: LSTM without Attention
inputs_no_attention = Input(shape=(timesteps, n_features))
lstm1_no_attention = LSTM(128, return_sequences=True)(inputs_no_attention)
dropout1_no_attention = Dropout(0.3)(lstm1_no_attention)
lstm2_no_attention = LSTM(64)(dropout1_no_attention)
dropout2_no_attention = Dropout(0.3)(lstm2_no_attention)
dense1_no_attention = Dense(64, activation='relu')(dropout2_no_attention)
outputs_no_attention = Dense(1, activation='linear')(dense1_no_attention)

model_without_attention = Model(inputs=inputs_no_attention, outputs=outputs_no_attention)
model_without_attention.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')
model_without_attention.fit(X_train, y_train, epochs=150, batch_size=64, validation_data=(X_val, y_val),
                           callbacks=[EarlyStopping(patience=20, restore_best_weights=True)], verbose=0)

# Model 3: Transformer-based Model
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = Dropout(dropout)(x)
    x = Add()([inputs, x])
    x = LayerNormalization(epsilon=1e-6)(x)
    ff = Dense(ff_dim, activation="relu")(x)
    ff = Dense(inputs.shape[-1])(ff)
    ff = Dropout(dropout)(ff)
    x = Add()([x, ff])
    x = LayerNormalization(epsilon=1e-6)(x)
    return x

inputs_transformer = Input(shape=(timesteps, n_features))
x = transformer_encoder(inputs_transformer, head_size=64, num_heads=4, ff_dim=128, dropout=0.3)
x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.3)
x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.3)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = Dense(64, activation='relu')(x)
outputs_transformer = Dense(1, activation='linear')(x)

model_transformer = Model(inputs=inputs_transformer, outputs=outputs_transformer)
model_transformer.compile(optimizer=Adam(learning_rate=0.00005), loss='mse')
model_transformer.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_val, y_val),
                     callbacks=[EarlyStopping(patience=30, restore_best_weights=True)], verbose=0)

# Performance Evaluation
y_val_pred_attention = scaler_y.inverse_transform(model_with_attention.predict(X_val, verbose=0)[0])
y_val_pred_no_attention = scaler_y.inverse_transform(model_without_attention.predict(X_val, verbose=0))
y_val_pred_transformer = scaler_y.inverse_transform(model_transformer.predict(X_val, verbose=0))
y_val_true = scaler_y.inverse_transform(y_val)

mae_attention = mean_absolute_error(y_val_true, y_val_pred_attention)
mse_attention = mean_squared_error(y_val_true, y_val_pred_attention)
mae_no_attention = mean_absolute_error(y_val_true, y_val_pred_no_attention)
mse_no_attention = mean_squared_error(y_val_true, y_val_pred_no_attention)
mae_transformer = mean_absolute_error(y_val_true, y_val_pred_transformer)
mse_transformer = mean_squared_error(y_val_true, y_val_pred_transformer)

# Thresholds for failure detection
thresholds = {
    "Vibration_Hz": {"min": 5, "max": 30},
    "Engine_Thrust_kN": {"min": 1000, "max": 2000},
    "Fuel_Pump_Pressure_bar": {"min": 80, "max": 180},
    "Temperature_C": {"min": 20, "max": 80},
    "Operating_Hours": {"min": 0, "max": 800},
    "Turbine_Speed_rpm": {"min": 5000, "max": 15000},
    "Oil_Pressure_bar": {"min": 2, "max": 5},
    "Exhaust_Gas_Temp_C": {"min": 800, "max": 1200},
    "Fuel_Flow_Rate_Ls": {"min": 10, "max": 50},
    "Structural_Stress_MPa": {"min": 200, "max": 500}
}

# Prediction function
def predict_maintenance(vib, thrust, pressure, temp, hours, turbine_speed, oil_pressure, exhaust_temp, fuel_flow, stress, subscription_tier="Basic"):
    try:
        # Clamp inputs to realistic ranges
        inputs = [
            min(max(vib, 5), 5000),
            min(max(thrust, 200), 2000),
            min(max(pressure, 50), 200),
            min(max(temp, 20), 200),
            min(max(hours, 0), 1000),
            min(max(turbine_speed, 200), 15000),
            min(max(oil_pressure, 2), 10000),
            min(max(exhaust_temp, 800), 2000),
            min(max(fuel_flow, 5), 50),
            min(max(stress, 200), 1000)
        ]

        raw_inputs = [vib, thrust, pressure, temp, hours, turbine_speed, oil_pressure, exhaust_temp, fuel_flow, stress]

        # Simulate 10 recent timesteps
        input_data = np.zeros((1, 10, n_features))
        for t in range(10):
            input_data[0, t] = [
                min(max(inputs[0] + np.random.uniform(-1, 1) * (t/10), 5), 5000),
                min(max(inputs[1] - np.random.uniform(0, 10) * (t/10), 200), 2000),
                min(max(inputs[2] + np.random.uniform(-3, 3) * (t/10), 50), 200),
                min(max(inputs[3] + np.random.uniform(0, 3) * (t/10), 20), 200),
                min(max(inputs[4] + t * 10, 0), 1000),
                min(max(inputs[5] - np.random.uniform(0, 50) * (t/10), 200), 15000),
                min(max(inputs[6] - np.random.uniform(0, 0.05) * (t/10), 2), 10000),
                min(max(inputs[7] + np.random.uniform(0, 10) * (t/10), 800), 2000),
                min(max(inputs[8] - np.random.uniform(0, 0.5) * (t/10), 5), 50),
                min(max(inputs[9] + np.random.uniform(0, 3) * (t/10), 200), 1000)
            ]

        # Pad with realistic trends to match timesteps
        padded_input = np.zeros((1, timesteps, n_features))
        for t in range(timesteps):
            if t < 90:
                padded_input[0, t] = [
                    min(max(inputs[0] + np.random.uniform(-0.5, 0.5) * (t/90), 5), 5000),
                    min(max(inputs[1] - np.random.uniform(0, 3) * (t/90), 200), 2000),
                    min(max(inputs[2] + np.random.uniform(-1, 1) * (t/90), 50), 200),
                    min(max(inputs[3] + np.random.uniform(0, 1) * (t/90), 20), 200),
                    min(max(inputs[4] + t * 10, 0), 1000),
                    min(max(inputs[5] - np.random.uniform(0, 30) * (t/90), 200), 15000),
                    min(max(inputs[6] - np.random.uniform(0, 0.02) * (t/90), 2), 10000),
                    min(max(inputs[7] + np.random.uniform(0, 5) * (t/90), 800), 2000),
                    min(max(inputs[8] - np.random.uniform(0, 0.2) * (t/90), 5), 50),
                    min(max(inputs[9] + np.random.uniform(0, 1) * (t/90), 200), 1000)
                ]
            else:
                padded_input[0, t] = input_data[0, t - 90]

        # Normalize
        padded_scaled = np.array([scaler_X.transform(padded_input[0])])

        # Predict TTF
        ttf_scaled_attention, attn_weights = model_with_attention.predict(padded_scaled, verbose=0)
        ttf_hours_attention = scaler_y.inverse_transform(ttf_scaled_attention)[0][0]
        ttf_hours_attention = max(0, ttf_hours_attention)

        ttf_scaled_no_attention = model_without_attention.predict(padded_scaled, verbose=0)
        ttf_hours_no_attention = scaler_y.inverse_transform(ttf_scaled_no_attention)[0][0]
        ttf_hours_no_attention = max(0, ttf_hours_no_attention)

        ttf_scaled_transformer = model_transformer.predict(padded_scaled, verbose=0)
        ttf_hours_transformer = scaler_y.inverse_transform(ttf_scaled_transformer)[0][0]
        ttf_hours_transformer = max(0, ttf_hours_transformer)

        # Check for threshold violations
        failure_causes = []
        for param, value in zip(thresholds.keys(), raw_inputs):
            thresh = thresholds[param]
            if "min" in thresh and value < thresh["min"]:
                failure_causes.append(f"{param} ({value:.1f} < {thresh['min']})")
            if "max" in thresh and value > thresh["max"]:
                failure_causes.append(f"{param} ({value:.1f} > {thresh['max']})")

        # Plot parameter trends
        time_steps = list(range(-90, 10))
        trends = [[] for _ in range(n_features)]
        for t in range(-90, 10):
            idx = max(0, min(timesteps - 1, t + 90))
            trends[0].append(padded_input[0, idx][0])
            trends[1].append(padded_input[0, idx][1])
            trends[2].append(padded_input[0, idx][2])
            trends[3].append(padded_input[0, idx][3])
            trends[4].append(padded_input[0, idx][4])
            trends[5].append(padded_input[0, idx][5])
            trends[6].append(padded_input[0, idx][6])
            trends[7].append(padded_input[0, idx][7])
            trends[8].append(padded_input[0, idx][8])
            trends[9].append(padded_input[0, idx][9])

        fig_trends = go.Figure()
        param_names = list(thresholds.keys())
        for i in range(n_features):
            fig_trends.add_trace(go.Scatter(x=time_steps, y=trends[i], mode='lines+markers', name=param_names[i]))
        fig_trends.update_layout(
            title="Parameter Trends (Last 100 Hours)",
            xaxis_title="Time (hours ago)",
            yaxis_title="Value",
            template="plotly_dark",
            legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5)
        )

        # Plot attention weights (Standard and Premium tiers only)
        fig_attention = None
        if subscription_tier in ["Standard", "Premium"]:
            fig_attention = go.Figure()
            attn_values = attn_weights[0, -100:, 0]
            fig_attention.add_trace(go.Scatter(x=time_steps, y=attn_values, mode='lines+markers', name='Attention Weights'))
            fig_attention.update_layout(
                title="Attention Weights Over Last 100 Timesteps",
                xaxis_title="Time (hours ago)",
                yaxis_title="Attention Weight",
                template="plotly_dark"
            )

        # Generate insights
        insights = f"### Model Performance on Validation Set\n"
        insights += f"- **LSTM with Attention**: MAE = {mae_attention:.2f} hours, MSE = {mse_attention:.2f}\n"
        insights += f"- **LSTM without Attention**: MAE = {mae_no_attention:.2f} hours, MSE = {mse_no_attention:.2f}\n"
        insights += f"- **Transformer**: MAE = {mae_transformer:.2f} hours, MSE = {mse_transformer:.2f}\n\n"

        insights += f"### Predictions for Current Input\n"
        insights += f"- **LSTM with Attention**: Predicted TTF = {ttf_hours_attention:.0f} hours\n"
        insights += f"- **LSTM without Attention**: Predicted TTF = {ttf_hours_no_attention:.0f} hours\n"
        insights += f"- **Transformer**: Predicted TTF = {ttf_hours_transformer:.0f} hours\n\n"

        insights += "Current Recorded Data:\n"
        for i, (param, value) in enumerate(zip(thresholds.keys(), raw_inputs)):
            insights += f"- {param}: {value:.1f}\n"

        # Extreme value warnings
        extreme_warnings = []
        for param, value in zip(thresholds.keys(), raw_inputs):
            thresh = thresholds[param]
            if ("min" in thresh and value < thresh["min"] * 0.1) or ("max" in thresh and value > thresh["max"] * 10):
                extreme_warnings.append(f"{param} ({value:.1f}) is far outside expected range ({thresh.get('min', 'N/A')}-{thresh.get('max', 'N/A')})")
        if extreme_warnings:
            insights += "⚠️ Warning: The following inputs are extremely out of range and may affect prediction accuracy:\n"
            for warning in extreme_warnings:
                insights += f"- {warning}\n"

        # Attention analysis (Standard and Premium tiers only)
        if subscription_tier in ["Standard", "Premium"]:
            insights += "\n### Attention Analysis\n"
            max_attention_idx = np.argmax(attn_values)
            max_attention_time = time_steps[max_attention_idx]
            max_attention_value = attn_values[max_attention_idx]
            insights += f"- The highest attention weight ({max_attention_value:.4f}) occurs at {max_attention_time} hours ago.\n"
            insights += "  Parameters at this timestep:\n"
            for i, param in enumerate(thresholds.keys()):
                param_value = trends[i][max_attention_idx]
                insights += f"  - {param}: {param_value:.1f}\n"

        # Placeholder for real-time monitoring (Premium tier only)
        if subscription_tier == "Premium":
            insights += "\n### Real-Time Monitoring\n"
            insights += "Real-time monitoring is not yet implemented. Contact support to enable this feature.\n"

        # Status based on LSTM with Attention
        if ttf_hours_attention < 50 or len(failure_causes) >= 3:
            insights += "\n🚨 ALERT: Failure imminent! Immediate maintenance recommended."
            if failure_causes:
                insights += f"\nLikely Causes: {', '.join(failure_causes)}"
        elif ttf_hours_attention < 200 or len(failure_causes) > 0:
            insights += "\n⚠️ Warning: Schedule maintenance soon."
            if failure_causes:
                insights += f"\nLikely Causes: {', '.join(failure_causes)}"
        else:
            insights += "\n✅ Equipment healthy"

        return insights, fig_trends, fig_attention

    except Exception as e:
        error_msg = f"Error in predict_maintenance: {str(e)}\n{traceback.format_exc()}"
        return error_msg, None, None

# Gradio interface
interface = gr.Interface(
    fn=predict_maintenance,
    inputs=[
        gr.Number(label="Vibration (Hz)", value=20),
        gr.Number(label="Engine Thrust (kN)", value=1500),
        gr.Number(label="Fuel Pump Pressure (bar)", value=100),
        gr.Number(label="Temperature (°C)", value=50),
        gr.Number(label="Operating Hours", value=500),
        gr.Number(label="Turbine Speed (rpm)", value=10000),
        gr.Number(label="Oil Pressure (bar)", value=3.5),
        gr.Number(label="Exhaust Gas Temp (°C)", value=900),
        gr.Number(label="Fuel Flow Rate (L/s)", value=30),
        gr.Number(label="Structural Stress (MPa)", value=300),
        gr.Dropdown(label="Subscription Tier", choices=["Basic", "Standard", "Premium"], value="Basic")
    ],
    outputs=[
        gr.Textbox(label="Maintenance Insights"),
        gr.Plot(label="Parameter Trends"),
        gr.Plot(label="Attention Weights")
    ],
    title="Predictive Maintenance Solutions PoC - Phase 1 (Sentinel Space)",
    description="Enter current equipment data to predict time to failure. Select your subscription tier to access advanced features. Part of Sentinel Space's Phase 1 initiative."
)

interface.launch(share=True)  # Generates a public URL for demo

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8131c55355102cf0bc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
